In [21]:
import os
import json

import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from tqdm.notebook import tqdm

In [22]:
load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")

In [4]:
isic_df = pd.read_excel('data/isic5codes.xlsx')
isic_df.head()

,ISIC5_Section,CAT,ISIC5_Code,ISIC5_Title,ISIC5_Desc
0,A,A,A,"Agriculture, forestry and fishing",This section includes the exploitation of vege...
1,A01,A,01,"Crop and animal production, hunting and relate...","This division includes two basic activities, n..."
2,A011,A,011,Growing of non-perennial crops,This group includes the growing of non-perenni...
3,A0111,A,0111,"Growing of cereals (except rice), leguminous c...",This class includes all forms of growing of ce...
4,A0112,A,0112,Growing of rice,This class includes:\n- growing of rice\n


In [3]:
nace_df = pd.read_excel('data/nace21codes.xlsx')
nace_df.head()

,ID,CAT,CODE,HEADING,PARENT_ID,PARENT_CODE,LEVEL,DESC
0,A,A,A,"AGRICULTURE, FORESTRY AND FISHING",NaN,NaN,1,This section includes the exploitation of vege...
1,01,A,01,"Crop and animal production, hunting and relate...",A,A,2,"This division includes two basic activities, n..."
2,011,A,01.1,Growing of non-perennial crops,01,01,3,This group includes the growing of non-perenni...
3,0111,A,01.11,"Growing of cereals, other than rice, leguminou...",011,01.1,4,This class includes all forms of growing of ce...
4,0112,A,01.12,Growing of rice,011,01.1,4,This class includes:\n- growing of rice


# Check if code works

In [19]:
def create_code_description(df, system_name):
    code_col = "ISIC5_Code" if system_name == "ISIC" else "CODE"
    title_col = "ISIC5_Title" if system_name == "ISIC" else "HEADING"
    desc_col = "ISIC5_Desc" if system_name == "ISIC" else "DESC"
    return [f"{system_name} Code: {row[code_col]} \n Title: {row[title_col]} \n Description: {row[desc_col]} \n" for _, row in df.iterrows()]

cat_prompted = ['A', 'B']
isic_descriptions = create_code_description(isic_df[isic_df['CAT'].isin(cat_prompted)], "ISIC")
nace_descriptions = create_code_description(nace_df[nace_df['CAT'].isin(cat_prompted)], "NACE")

In [11]:
isic_descriptions[:5]

['ISIC Code: A \n Title: Agriculture, forestry and fishing \n Description: This section includes the exploitation of vegetal and animal natural resources, comprising the activities of growing of crops, raising and breeding of animals, harvesting of timber and other plants, production of animal products from a farm or natural habitats.\nThis section includes also:\norganic agriculture, soilless cultivation of crops, hydroponics and aquaponics, aquaculture, the growing of genetically modified crops and the raising of genetically modified animals\nThis section excludes undifferentiated subsistence goods-producing activities of households, which is classified in class 9810. \n',
 'ISIC Code: 01 \n Title: Crop and animal production, hunting and related service activities \n Description: This division includes two basic activities, namely the production of crop products and production of _x000D_animal products. This division includes growing of crops on open land, under cover or in greenhous

In [ ]:
prompt = f"""
I need to create a correspondence mapping between two industrial classification systems: ISIC (International Standard Industrial Classification) and NACE (Statistical Classification of Economic Activities in the European Community EU). This is a new classification version so the correspondence is not readily available. Since there is a token limit, I will provide you with the data in parts.

Here are the ISIC codes with their titles and descriptions:
{chr(10).join(isic_descriptions)}

Here are the NACE codes with their titles and descriptions:
{chr(10).join(nace_descriptions)}

Please analyze these classification systems and provide a mapping between them. For each mapping, indicate:
1. The ISIC code(s) and NACE code(s) that correspond
2. Confidence level (High/Medium/Low)
3. Type of match (one-to-one, one-to-many, many-to-one, many-to-many)

Return the results in a structured JSON format like:
{{
  "mappings": [
    {{
      "isic_codes": ["A"],
      "nace_codes": ["A"],
      "confidence": "High",
      "match_type": "one-to-one"
    }},
    {{
      "isic_codes": ["0141"],
      "nace_codes": ["01.41", "01.42"],
      "confidence": "High",
      "match_type": "one-to-many"
    }}
  ],
  "unmatched_codes": {{
    "isic": ["05", "10", "9000"],
    "nace": ["05", "90.11"]
  }}
}}

Important matching principles:
1. Check hierarchical relationships
2. Look for semantic similarity in titles and descriptions
3. Consider cross-references between systems
4. Note that NACE is based on ISIC but has more granular European adaptations
5. Avoid suggesting matches for codes not provided in the input data
6. Only map codes of the same level (e.g., 4-digit to 4-digit)
7. if NACE is more granular and ISIC is broader, try to map multiple NACE codes to a single ISIC code; provided that no other ISIC code matches those NACE codes
"""

In [13]:
def estimate_tokens(text):
    """
    Rough estimation: tokens ≈ words/0.75 or chars/4
    Accurate for English, approximate for code/mixed content
    """
    # Simple word-based estimation
    word_count = len(text.split())
    token_estimate = int(word_count / 0.75)
    
    # Alternative: character-based estimation
    char_count = len(text)
    token_estimate_char = int(char_count / 4)
    
    return max(token_estimate, token_estimate_char)

estimated_tokens = estimate_tokens(prompt)
estimated_tokens

23576

In [23]:
client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com"
)


In [15]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.1,  # Low temperature for consistent output
    max_tokens=4000
)

In [70]:
print(response.choices[0].message.content)

Looking at the provided data for Sections A and B, I can create a detailed mapping. The structures are extremely similar, with NACE being a more granular European adaptation of ISIC. The main differences are in the formatting of codes (ISIC uses 4-digit codes without decimals, NACE uses a decimal system) and some minor title variations and splits.

Here is the structured JSON mapping:

```json
{
  "mappings": [
    {
      "isic_codes": ["A"],
      "nace_codes": ["A"],
      "confidence": "High",
      "match_type": "one-to-one"
    },
    {
      "isic_codes": ["01"],
      "nace_codes": ["01"],
      "confidence": "High",
      "match_type": "one-to-one"
    },
    {
      "isic_codes": ["011"],
      "nace_codes": ["01.1"],
      "confidence": "High",
      "match_type": "one-to-one"
    },
    {
      "isic_codes": ["0111"],
      "nace_codes": ["01.11"],
      "confidence": "High",
      "match_type": "one-to-one"
    },
    {
      "isic_codes": ["0112"],
      "nace_codes": ["0

In [16]:
def save_response_basic(response_content, filename, folder="raw-output"):
    """Save response content to a text file"""
    
    # Create folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)
    # Full file path
    filepath = os.path.join(folder, filename)
    
    # Save the content
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(response_content)
    
    print(f"✅ Response saved to: {filepath}")
    return filepath


In [17]:
nametxt = ''.join(cat_prompted)
save_response_basic(response.choices[0].message.content, filename=f"{nametxt}.md")

✅ Response saved to: raw-output\AB.md


'raw-output\\AB.md'

# Real Batch Process

In [ ]:
def create_prompt(isic_desc, nace_desc):
    return f"""
I need to create a correspondence mapping between two industrial classification systems: ISIC (International Standard Industrial Classification) and NACE (Statistical Classification of Economic Activities in the European Community EU). This is a new classification version so the correspondence is not readily available. Since there is a token limit, I will provide you with the data in parts.

Here are the ISIC codes with their titles and descriptions:
{chr(10).join(isic_desc)}

Here are the NACE codes with their titles and descriptions:
{chr(10).join(nace_desc)}

Please analyze these classification systems and provide a mapping between them. For each mapping, indicate:
1. The ISIC code(s) and NACE code(s) that correspond
2. Confidence level (High/Medium/Low)
3. Type of match (one-to-one, one-to-many, many-to-one, many-to-many)

Return the results in a structured JSON format like:
{{
  "mappings": [
    {{
      "isic_codes": ["A"],
      "nace_codes": ["A"],
      "confidence": "High",
      "match_type": "one-to-one"
    }},
    {{
      "isic_codes": ["0141"],
      "nace_codes": ["01.41", "01.42"],
      "confidence": "High",
      "match_type": "one-to-many"
    }}
  ],
  "unmatched_codes": {{
    "isic": ["05", "10", "9000"],
    "nace": ["05", "90.11"]
  }}
}}

Important matching principles:
1. Check hierarchical relationships
2. Look for semantic similarity in titles and descriptions
3. Consider cross-references between systems
4. Note that NACE is based on ISIC but has more granular European adaptations
5. Avoid suggesting matches for codes not provided in the input data
6. Only map codes of the same level (e.g., 4-digit to 4-digit)
7. if NACE is more granular and ISIC is broader, try to map multiple NACE codes to a single ISIC code; provided that no other ISIC code matches those NACE codes
"""

In [21]:
cat_prompted_list = [
    ['A', 'B'],
    ['C'],
    ['D', 'E', 'F'],
    ['G'],
    ['H', 'I'],
    ['J', 'K'],
    ['L', 'M', 'N'],
    ['O', 'P'],
    ['Q', 'R', 'S'],
    ['T', 'U', 'V']
]

for cat_prompted in tqdm(cat_prompted_list):
    isic_descriptions = create_code_description(isic_df[isic_df['CAT'].isin(cat_prompted)], "ISIC")
    nace_descriptions = create_code_description(nace_df[nace_df['CAT'].isin(cat_prompted)], "NACE")
    prompt = create_prompt(isic_descriptions, nace_descriptions)
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,  # Low temperature for consistent output
        max_tokens=8000
    )
    print(response.usage)
    
    nametxt = ''.join(cat_prompted)
    save_response_basic(response.choices[0].message.content, filename=f"{nametxt}.md")

  0%|          | 0/10 [00:00<?, ?it/s]

CompletionUsage(completion_tokens=3672, prompt_tokens=22586, total_tokens=26258, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=22528), prompt_cache_hit_tokens=22528, prompt_cache_miss_tokens=58)
✅ Response saved to: raw-output\AB.md
CompletionUsage(completion_tokens=8000, prompt_tokens=103093, total_tokens=111093, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=103040), prompt_cache_hit_tokens=103040, prompt_cache_miss_tokens=53)
✅ Response saved to: raw-output\C.md
CompletionUsage(completion_tokens=2234, prompt_tokens=19137, total_tokens=21371, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=19136), prompt_cache_hit_tokens=19136, prompt_cache_miss_tokens=1)
✅ Response saved to: raw-output\DEF.md
CompletionUsage(completion_tokens=3143, prompt_tokens=26077, total_tokens=29220, completion_tokens_details=None, pr

# Edit JSON output
- only keep 1 and 4 digits code; remove 2 and 3 digits since the AI mixing 3 and 4 digits in a single record
- rebuilding the 2 and 3 digits can be done from the 4 digits
- if needed clean up manually from `isic-nace.json` then run again the process

In [27]:
# Load the JSON file
with open('processed-output/isic-nace.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Clean up the data
cleaned_data = []
for record in data:
    # Filter isic_codes: keep only items with length NOT in [2, 3]
    cleaned_isic = [code for code in record['isic_codes'] if len(code) not in [2, 3]]
    
    # Filter nace_codes: keep only items with length NOT in [2, 3, 4]
    cleaned_nace = [code for code in record['nace_codes'] if len(code) not in [2, 3, 4]]
    
    # Only keep the record if at least one of the lists is not empty
    if cleaned_isic or cleaned_nace:
        cleaned_data.append({
            'isic_codes': cleaned_isic,
            'nace_codes': cleaned_nace,
            'confidence': record['confidence'],
            'match_type': record['match_type']
        })

# Save the cleaned data
with open('processed-output/isic-nace-cleaned.json', 'w', encoding='utf-8') as f:
    json.dump(cleaned_data, f, indent=2, ensure_ascii=False)

print(f"Original records: {len(data)}")
print(f"Cleaned records: {len(cleaned_data)}")
print(f"Removed {len(data) - len(cleaned_data)} records")

Original records: 691
Cleaned records: 473
Removed 218 records


In [28]:
# Get all 4-digit codes from master data
isic_master_4digit = set(isic_df[isic_df['ISIC5_Code'].str.len() == 4]['ISIC5_Code'])
nace_master_4digit = set(nace_df[nace_df['CODE'].str.len() == 5]['CODE'])  # NACE 4-digit is like "01.11" (5 chars)

print(f"ISIC master 4-digit codes: {len(isic_master_4digit)}")
print(f"NACE master 4-digit codes: {len(nace_master_4digit)}")

# Get all codes from cleaned JSON
with open('processed-output/isic-nace-cleaned.json', 'r', encoding='utf-8') as f:
    cleaned_data = json.load(f)

json_isic_codes = set()
json_nace_codes = set()

for record in cleaned_data:
    json_isic_codes.update(record['isic_codes'])
    json_nace_codes.update(record['nace_codes'])

# Filter to 4-digit only
json_isic_4digit = {code for code in json_isic_codes if len(code) == 4}
json_nace_4digit = {code for code in json_nace_codes if len(code) == 5}

print(f"\nJSON ISIC 4-digit codes: {len(json_isic_4digit)}")
print(f"JSON NACE 4-digit codes: {len(json_nace_4digit)}")

# Find missing codes
missing_isic = isic_master_4digit - json_isic_4digit
missing_nace = nace_master_4digit - json_nace_4digit

print(f"\n❌ Missing ISIC 4-digit codes: {len(missing_isic)}")
if missing_isic:
    print(sorted(missing_isic))

print(f"\n❌ Missing NACE 4-digit codes: {len(missing_nace)}")
if missing_nace:
    print(sorted(missing_nace))

# Extra codes (in JSON but not in master)
extra_isic = json_isic_4digit - isic_master_4digit
extra_nace = json_nace_4digit - nace_master_4digit

print(f"\n⚠️ Extra ISIC codes in JSON: {len(extra_isic)}")
if extra_isic:
    print(sorted(extra_isic))

print(f"\n⚠️ Extra NACE codes in JSON: {len(extra_nace)}")
if extra_nace:
    print(sorted(extra_nace))

ISIC master 4-digit codes: 463
NACE master 4-digit codes: 651

JSON ISIC 4-digit codes: 463
JSON NACE 4-digit codes: 651

❌ Missing ISIC 4-digit codes: 0

❌ Missing NACE 4-digit codes: 0

⚠️ Extra ISIC codes in JSON: 0

⚠️ Extra NACE codes in JSON: 0


In [25]:
# ask AI for remaining leftover in NACE codes, luckily only category C has leftovers
isic_descriptions = create_code_description(isic_df[isic_df['CAT'].isin(['C'])], "ISIC")
nace_descriptions = create_code_description(nace_df[nace_df['CAT'].isin(['C'])], "NACE")

prompt = f"""
TASK: Map the remaining UNMAPPED NACE codes to their most appropriate ISIC codes.

BACKGROUND:
- You previously mapped many NACE codes to ISIC codes, but some codes remain unmapped.
- These NACE codes might be: 
  1. More granular than ISIC equivalents
  2. European-specific classifications not in ISIC
  3. Codes you may have missed in initial analysis
  4. New or specialized economic activities

UNMAPPED NACE CODES:
['10.52', '24.20', '24.31', '24.32', '24.33', '24.34', '25.12', '28.14']

RELEVANT ISIC CODES (for context):
{chr(10).join(isic_descriptions)}

RELEVANT NACE CODES (for context):
{chr(10).join(nace_descriptions)}



Start by identifying obvious matches, then tackle difficult cases.Please analyze these classification systems and provide a mapping between them. For each mapping, indicate:
1. The ISIC code(s) and NACE code(s) that correspond
2. Confidence level (High/Medium/Low)
3. Type of match (one-to-one, one-to-many, many-to-one, many-to-many)

Return the results in a structured JSON format like:
{{
  "mappings": [
    {{
      "isic_codes": ["A"],
      "nace_codes": ["A"],
      "confidence": "High",
      "match_type": "one-to-one"
    }},
    {{
      "isic_codes": ["0141"],
      "nace_codes": ["01.41", "01.42"],
      "confidence": "High",
      "match_type": "one-to-many"
    }}
  ],
  "unmatched_codes": {{
    "nace": ["05", "90.11"]
  }}
}}

Focus on accuracy over speed.
"""
    
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.1,  # Low temperature for consistent output
    max_tokens=4000
)

In [26]:
print(response.choices[0].message.content)

Based on my analysis of the NACE and ISIC classification systems, here is the mapping for the remaining unmapped NACE codes.

```json
{
  "mappings": [
    {
      "isic_codes": ["1050"],
      "nace_codes": ["10.52"],
      "confidence": "High",
      "match_type": "one-to-one",
      "notes": "NACE 10.52 'Manufacture of ice cream and other edible ice' is a direct subset of ISIC 1050 'Manufacture of dairy products', which explicitly includes 'manufacture of ice cream'."
    },
    {
      "isic_codes": ["2410"],
      "nace_codes": ["24.20"],
      "confidence": "High",
      "match_type": "one-to-one",
      "notes": "NACE 24.20 'Manufacture of tubes, pipes, hollow profiles and related fittings, of steel' is a specific product line within the broader ISIC 2410 'Manufacture of basic iron and steel', which includes the manufacture of seamless and welded tubes and pipes of steel."
    },
    {
      "isic_codes": ["2410"],
      "nace_codes": ["24.31"],
      "confidence": "High",
     

In [8]:
df = pd.read_json('processed-output/isic-nace-cleaned.json')
df.to_excel('processed-output/isic-nace.xlsx', index=False)